# 🎤 Agent Vocal IA avec RAG Agentique + Pipecat
## 100% Local | Streaming Temps Réel | Google Colab

---

### 📋 Vue d'ensemble du projet

Ce notebook présente un **agent vocal IA intelligent** utilisant **Pipecat** pour le streaming temps réel :

- 🎙️ **Reconnaissance vocale** (Whisper via faster-whisper)
- 🤖 **LLM local streaming** (Ollama - Qwen2 1.5B)
- 📚 **RAG Agentique** (ChromaDB/FAISS avec routing)
- 🔊 **Synthèse vocale** (Piper TTS français)
- ⚡ **Pipeline Pipecat** (latence <2s)

### 🎯 Domaines supportés
- **🔢 Mathématiques** : Équations, algèbre, géométrie
- **⚛️ Physique** : Mécanique, lois de Newton
- **🇬🇧 Anglais** : Grammaire, temps verbaux

### 🏗️ Architecture Pipecat
```
Audio Input → LocalSTTService → RAGService → LocalLLMService → LocalTTSService → Audio Output
              (Whisper)        (Router+RAG)  (Ollama)         (Piper)
```

---

**⚠️ IMPORTANT** : Activez le GPU :
- `Runtime → Change runtime type → Hardware accelerator → GPU (T4)`

**📊 Temps d'installation** : ~10-12 minutes

## 📦 Étape 1 : Vérification GPU et Installation des Dépendances

Installation de tous les packages nécessaires (~5 minutes)

In [ ]:
import sys
import subprocess
import os

print("🚀 Vérification de l'environnement...")
print("=" * 70)

# Vérifier GPU
!nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv

print("\n✅ GPU détecté !")
print("\n📦 Installation des dépendances Python...")
print("=" * 70)

# Fix opentelemetry conflicts first
print("\n🔧 Résolution des conflits de dépendances...")
!pip install -q --upgrade opentelemetry-api==1.37.0 opentelemetry-sdk==1.37.0 opentelemetry-proto==1.37.0

# Install core dependencies
print("\n📦 Installation de Pipecat et dépendances...")
!pip install -q pipecat-ai[silero]
!pip install -q faster-whisper
!pip install -q sentence-transformers
!pip install -q chromadb faiss-cpu
!pip install -q piper-tts
!pip install -q gradio
!pip install -q soundfile librosa

print("\n✅ Toutes les dépendances Python sont installées !")
print("ℹ️  Les warnings opentelemetry sont normaux et n'affectent pas notre agent")

## 🔧 Étape 2 : Installation et Démarrage d'Ollama

Ollama est nécessaire pour exécuter le LLM localement (~3 minutes)

In [ ]:
print("🤖 Installation d'Ollama...")
print("=" * 70)

# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

print("\n✅ Ollama installé !")

# Start Ollama server in background
print("\n🚀 Démarrage du serveur Ollama...")
import subprocess
import time

# Start ollama serve in background
ollama_process = subprocess.Popen(['ollama', 'serve'], 
                                   stdout=subprocess.DEVNULL, 
                                   stderr=subprocess.DEVNULL)

# Wait for server to start
time.sleep(5)

print("✅ Serveur Ollama démarré !")

# Pull the model
print("\n📥 Téléchargement du modèle Qwen2 1.5B (optimisé pour Colab)...")
print("⏳ Cela peut prendre 2-3 minutes...")
!ollama pull qwen2:1.5b

print("\n✅ Modèle Qwen2 1.5B prêt !")

# Verify
!ollama list

## 📂 Étape 3 : Clonage du Repository et Setup

Clonage du code source depuis GitHub

In [ ]:
import os
from pathlib import Path

print("📂 Clonage du repository...")
print("=" * 70)

# Clone repository
repo_url = "https://github.com/Romainmlt123/agent-vocal-ia-RAG-Agentique.git"
branch = "pipecat-local-colab"
repo_name = "agent-vocal-ia-RAG-Agentique"

if not os.path.exists(repo_name):
    !git clone -b {branch} {repo_url}
    print(f"✅ Repository cloné (branche: {branch})")
else:
    print(f"ℹ️  Repository déjà présent")

# Change to repo directory
os.chdir(repo_name)
print(f"\n📁 Répertoire de travail : {os.getcwd()}")

# Add src to path
import sys
sys.path.insert(0, str(Path.cwd() / 'src'))

print("\n✅ Setup complet !")

## 📥 Étape 4 : Téléchargement des Modèles Whisper et Piper

Téléchargement des modèles pour STT et TTS

In [ ]:
print("📥 Téléchargement des modèles...")
print("=" * 70)

# Whisper will be downloaded automatically by faster-whisper on first use
print("✅ Whisper : sera téléchargé automatiquement au premier usage")

# Download Piper voice model
print("\n📥 Téléchargement de la voix Piper (fr_FR-siwis-medium)...")

import urllib.request
import os

os.makedirs("models/voices", exist_ok=True)

voice_url = "https://huggingface.co/rhasspy/piper-voices/resolve/main/fr/fr_FR/siwis/medium/fr_FR-siwis-medium.onnx"
config_url = "https://huggingface.co/rhasspy/piper-voices/resolve/main/fr/fr_FR/siwis/medium/fr_FR-siwis-medium.onnx.json"

voice_path = "models/voices/fr_FR-siwis-medium.onnx"
config_path = "models/voices/fr_FR-siwis-medium.onnx.json"

if not os.path.exists(voice_path):
    print("⏳ Téléchargement du modèle vocal...")
    urllib.request.urlretrieve(voice_url, voice_path)
    print("✅ Modèle vocal téléchargé")
else:
    print("ℹ️  Modèle vocal déjà présent")

if not os.path.exists(config_path):
    print("⏳ Téléchargement de la config...")
    urllib.request.urlretrieve(config_url, config_path)
    print("✅ Config téléchargée")
else:
    print("ℹ️  Config déjà présente")

print("\n✅ Tous les modèles sont prêts !")

## 🗄️ Étape 5 : Construction des Index RAG

Création des index vectoriels pour chaque domaine

In [ ]:
print("🗄️  Construction des index RAG...")
print("=" * 70)

# Utiliser le script standalone (pas de dépendance src.config/utils)
!python scripts/build_indexes_colab.py

print("\n✅ Index RAG construits !")

## 🎯 Étape 6 : Initialisation du Pipeline Pipecat

Création et initialisation du pipeline vocal complet

In [ ]:
print("🎯 Initialisation du pipeline Pipecat...")
print("=" * 70)

import asyncio
import logging

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Import pipeline
from src.pipeline.voice_pipeline import create_voice_pipeline

# Create pipeline
print("\n⏳ Création du pipeline (cela peut prendre 30-60 secondes)...\n")

pipeline = await create_voice_pipeline(
    whisper_model="base",      # tiny/base/small (base recommandé pour Colab T4)
    ollama_model="qwen2:1.5b",  # Modèle optimisé pour Colab
    device="cuda",              # GPU acceleration
    rag_data_path="data"        # Chemin vers les données RAG
)

print("\n" + "="*70)
print("✅ Pipeline Pipecat initialisé avec succès !")
print("="*70)
print("\n📊 Configuration :")
print(f"  • STT      : Whisper base (faster-whisper + CUDA)")
print(f"  • LLM      : Ollama Qwen2 1.5B")
print(f"  • TTS      : Piper fr_FR-siwis-medium")
print(f"  • RAG      : 3 domaines (maths, physique, anglais)")
print(f"  • Framework: Pipecat")
print(f"  • Device   : CUDA (GPU)")
print("\n🚀 Prêt pour le traitement !")

## 🧪 Étape 7 : Test Rapide du Pipeline

Test avec une question textuelle pour valider le fonctionnement

In [ ]:
print("🧪 Test du pipeline avec une question exemple...")
print("=" * 70)

test_question = "Comment résoudre une équation du second degré ?"
print(f"\n❓ Question : {test_question}")
print("\n⏳ Traitement en cours...\n")

# Process the question
result = await pipeline.process_text(test_question)

print("\n" + "="*70)
print("📊 RÉSULTATS")
print("="*70)
print(f"\n📝 Transcription : {result['transcription']}")
print(f"\n📚 Domaine détecté : {result['subject']}")
print(f"\n💡 Réponse :\n{result['response']}")
print(f"\n🔊 Audio généré : {len(result['audio_output'])} bytes à {result['sample_rate']} Hz")
print("\n✅ Test réussi ! Le pipeline fonctionne correctement.")

## 🎨 Étape 8 : Lancement de l'Interface Gradio

Interface graphique complète pour interagir avec l'agent vocal

In [ ]:
print("🎨 Lancement de l'interface Gradio...")
print("=" * 70)

from src.ui.ui_gradio_pipecat import create_gradio_app

# Create Gradio app with the pipeline
app = create_gradio_app(pipeline)

# Build and launch interface
app.build_interface()

print("\n🚀 Lancement de l'interface...")
print("\n" + "="*70)
print("✅ Interface Gradio en cours de démarrage...")
print("="*70)
print("\n📱 Un lien public va apparaître ci-dessous dans quelques secondes.")
print("   Cliquez sur ce lien pour accéder à l'interface !")
print("\n💡 Vous pouvez :")
print("   • Utiliser le microphone pour poser des questions vocalement")
print("   • Taper vos questions directement")
print("   • Écouter les réponses synthétisées")
print("\n" + "="*70)

app.launch(
    share=True,          # Create public link for Colab
    server_port=7860
)

## 📊 Informations Complémentaires

### Architecture Pipecat

Le pipeline est composé des processeurs suivants dans l'ordre :

1. **LocalSTTService** : Whisper (faster-whisper) pour la transcription
2. **TranscriptionCollector** : Collecte la transcription
3. **RAGService** : Détection du domaine + retrieval de documents
4. **LocalLLMService** : Ollama pour la génération de réponse
5. **ResponseCollector** : Collecte la réponse
6. **LocalTTSService** : Piper pour la synthèse vocale
7. **AudioBufferProcessor** : Collecte l'audio de sortie

### Performance Attendue (Colab T4)

| Composant | Latence |
|-----------|--------|
| STT (Whisper base) | ~200ms |
| RAG (retrieval + routing) | ~100ms |
| LLM (Qwen2 1.5B) | ~800ms |
| TTS (Piper) | ~300ms |
| **Total** | **~1.4s** |

### Avantages de Pipecat

- ✅ **Streaming** : Traitement asynchrone des frames
- ✅ **Modulaire** : Facile d'ajouter/remplacer des composants
- ✅ **Performant** : Pipeline optimisé pour la latence
- ✅ **Flexible** : Support audio + texte

### Utilisation

1. **Onglet "Entrée Vocale"** : Cliquez sur le micro, posez votre question
2. **Onglet "Entrée Texte"** : Tapez votre question ou utilisez les exemples
3. Attendez la réponse (1-2 secondes)
4. Écoutez la réponse vocale générée

### Questions Exemples

**Mathématiques** 🔢
- Comment résoudre une équation du second degré ?
- Explique-moi le théorème de Pythagore

**Physique** ⚛️
- Qu'est-ce que la force de gravitation ?
- Quelle est la troisième loi de Newton ?

**Anglais** 🇬🇧
- Comment conjuguer le verbe 'to be' au présent ?
- Comment utiliser le present perfect ?

---

**🎓 Projet réalisé par** : Romain Mallet  
**📅 Date** : Novembre 2024  
**🔗 Repository** : [GitHub](https://github.com/Romainmlt123/agent-vocal-ia-RAG-Agentique)